In [1]:
import numpy as np
import pandas as pd

In [2]:
def changecolumn(dataset, AttackType):
    df = pd.read_csv(dataset).sample(frac = 0.1, random_state = 20, replace = False).reset_index(drop=True)
    df.columns = ["Timestamp", "CAN ID", "Byte", "DATA[0]","DATA[1]","DATA[2]","DATA[3]","DATA[4]","DATA[5]","DATA[6]","DATA[7]","AttackType"]
    df['AttackType'] = np.where(df['AttackType'] == 'T',AttackType, 'R')
    df.dropna()
    return df

dfDos = changecolumn('DoS_dataset.csv','DoS Attack')
dfFuzzy = changecolumn('Fuzzy_dataset.csv','Fuzzy Attack')
dfGear = changecolumn('gear_dataset.csv','Gear Spooing Attack')
dfRPM = changecolumn('RPM_dataset.csv','RPM Spoofing Attack')
frames = [dfDos, dfFuzzy, dfGear, dfRPM]
df = pd.concat(frames)
# print(df.head(10))
# print(df.shape)

In [3]:
df

,Timestamp,CAN ID,Byte,DATA[0],DATA[1],DATA[2],DATA[3],DATA[4],DATA[5],DATA[6],DATA[7],AttackType
0,1.478200e+09,0000,8,00,00,00,00,00,00,00,00,DoS Attack
1,1.478201e+09,0131,8,1b,80,00,00,3f,7f,0e,a6,R
2,1.478199e+09,00a1,8,80,89,00,00,24,00,00,00,R
3,1.478200e+09,0260,8,18,21,22,30,08,8f,70,05,R
4,1.478201e+09,02c0,8,14,00,00,00,00,00,00,00,R
...,...,...,...,...,...,...,...,...,...,...,...,...
462165,1.478191e+09,0329,8,dc,b7,7e,14,11,20,00,14,R
462166,1.478192e+09,0131,8,f2,7f,00,00,3a,7f,0c,83,R
462167,1.478201e+09,0131,8,f2,7f,00,00,40,7f,06,16,R
462168,1.478193e+09,02c0,8,15,00,00,00,00,00,00,00,R


In [4]:
df.dtypes

Timestamp     float64
CAN ID         object
Byte            int64
DATA[0]        object
DATA[1]        object
DATA[2]        object
DATA[3]        object
DATA[4]        object
DATA[5]        object
DATA[6]        object
DATA[7]        object
AttackType     object
dtype: object

In [5]:
df = df.dropna()
def changecolumntype(df):
    for column in df[['CAN ID', 'DATA[0]', 'DATA[1]', 'DATA[2]', 'DATA[3]', 'DATA[4]', 'DATA[5]', 'DATA[6]', 'DATA[7]']]:
        df[column] = df[column].apply(lambda x: int(str(x), base=16))
    return df

df = changecolumntype(df)
# print(df.dtypes)
# df.head(10)

/tmp/ipykernel_14469/3871048372.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: int(str(x), base=16))


In [6]:
df.dtypes

Timestamp     float64
CAN ID          int64
Byte            int64
DATA[0]         int64
DATA[1]         int64
DATA[2]         int64
DATA[3]         int64
DATA[4]         int64
DATA[5]         int64
DATA[6]         int64
DATA[7]         int64
AttackType     object
dtype: object

In [7]:
# Remove two columns name "Timestamp"
df = df.drop(['Timestamp','Byte'], axis=1)

In [8]:
df

,CAN ID,DATA[0],DATA[1],DATA[2],DATA[3],DATA[4],DATA[5],DATA[6],DATA[7],AttackType
0,0,0,0,0,0,0,0,0,0,DoS Attack
1,305,27,128,0,0,63,127,14,166,R
2,161,128,137,0,0,36,0,0,0,R
3,608,24,33,34,48,8,143,112,5,R
4,704,20,0,0,0,0,0,0,0,R
...,...,...,...,...,...,...,...,...,...,...
462165,809,220,183,126,20,17,32,0,20,R
462166,305,242,127,0,0,58,127,12,131,R
462167,305,242,127,0,0,64,127,6,22,R
462168,704,21,0,0,0,0,0,0,0,R


In [9]:
attacktype = {'DoS Attack': 'Dos','Fuzzy Attack': 'Fuzzy','Gear Spooing Attack': 'gear','RPM Spoofing Attack': 'RPM'}

In [10]:
df.rename(columns = {'AttackType':'Label'}, inplace = True)
df.replace({"Label": attacktype}, inplace = True)

In [11]:
#saving the dataframe
df.to_csv('10_percentage_of_each_dataset.csv',index=False)